In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime as dt
from pandas.tseries.offsets import MonthEnd

In [3]:
# get ticket symbols for all S&P 500

In [4]:
df = pd.read_csv('nasdaq.csv')

In [5]:
tickers = df['Symbol'].tolist()

In [ ]:
# 2009 12 31, 2020 12 31
df_yf = yf.download(tickers, start='2009-05-01', end='2021-09-01')

[                       1%                       ]  15 of 2967 completed

Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\10455\anaconda3\envs\lgp\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\10455\anaconda3\envs\lgp\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\10455\anaconda3\envs\lgp\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\Users\10455\anaconda3\envs\lgp\lib\site-packages\yfinance\multi.py", line 170, in _download_one_threaded
    actions, period, interval, prepost, proxy, rounding)
  File "C:\Users\10455\anaconda3\envs\lgp\lib\site-packages\yfinance\multi.py", line 185, in _download_one
    rounding=rounding, many=True)
  File "C:\Users\10455\anaconda3\envs\lgp\lib\site-packages\yfinance\base.py", line 179, in history
    data = data.json()
  File "C:\Users\10455\anaconda3\envs\lgp\lib\site-packages\requests\models.py", line 898, in json
    return complexjs

[**                     5%                       ]  160 of 2967 completed

Exception in thread Thread-168:
Traceback (most recent call last):
  File "C:\Users\10455\anaconda3\envs\lgp\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\10455\anaconda3\envs\lgp\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\10455\anaconda3\envs\lgp\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\Users\10455\anaconda3\envs\lgp\lib\site-packages\yfinance\multi.py", line 170, in _download_one_threaded
    actions, period, interval, prepost, proxy, rounding)
  File "C:\Users\10455\anaconda3\envs\lgp\lib\site-packages\yfinance\multi.py", line 185, in _download_one
    rounding=rounding, many=True)
  File "C:\Users\10455\anaconda3\envs\lgp\lib\site-packages\yfinance\base.py", line 179, in history
    data = data.json()
  File "C:\Users\10455\anaconda3\envs\lgp\lib\site-packages\requests\models.py", line 898, in json
    return complex

[****                   9%                       ]  255 of 2967 completed

Exception in thread Thread-42:
Traceback (most recent call last):
  File "C:\Users\10455\anaconda3\envs\lgp\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\10455\anaconda3\envs\lgp\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\10455\anaconda3\envs\lgp\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\Users\10455\anaconda3\envs\lgp\lib\site-packages\yfinance\multi.py", line 170, in _download_one_threaded
    actions, period, interval, prepost, proxy, rounding)
  File "C:\Users\10455\anaconda3\envs\lgp\lib\site-packages\yfinance\multi.py", line 185, in _download_one
    rounding=rounding, many=True)
  File "C:\Users\10455\anaconda3\envs\lgp\lib\site-packages\yfinance\base.py", line 179, in history
    data = data.json()
  File "C:\Users\10455\anaconda3\envs\lgp\lib\site-packages\requests\models.py", line 898, in json
    return complexj

[*********************100%***********************]  2963 of 2967 completed

In [ ]:
df_yf

In [ ]:
prices = df_yf['Adj Close']

In [ ]:
prices

In [ ]:
prices.index = pd.to_datetime(prices.index)


In [ ]:
# track all monthly return
monthly_return = prices.pct_change().resample('M').agg(lambda x: (x+1).prod() -1)

In [ ]:
monthly_return

In [ ]:
past_11 = (monthly_return+1).rolling(11).apply(np.prod) - 1

In [ ]:
past_11

In [ ]:
formation = dt.datetime(2011, 5, 31)

In [ ]:
formation

In [ ]:
# calculate profit using momentum strategy
def momentum(formation):
    end_measurement = formation - MonthEnd(1) # from last month
    return_12 = past_11.loc[end_measurement]
    return_12 = return_12.reset_index()
    # separate monthly return into ten bins
    return_12['decile'] = pd.qcut(return_12.iloc[:,1], q = 10, labels=False, duplicates='drop')
    # get top winners and losers
    winners = return_12[return_12.decile == 9]
    losers = return_12[return_12.decile == 0]
    winner_return = monthly_return.loc[formation + MonthEnd(1), monthly_return.columns.isin(winners['index'])]
    loser_return = monthly_return.loc[formation + MonthEnd(1), monthly_return.columns.isin(losers['index'])]
    # profit = winner - loser
    momentum_profit = winner_return.mean() - loser_return.mean()
    return momentum_profit, return_12

profits = []
dates = []
for i in range(12*10): # for ten years
    date = formation + MonthEnd(i)
    p, _ = momentum(date)
    profits.append(p)
    dates.append(date)
# list containing profit for each month, 10 years
profits


In [ ]:
# check info in a particular month
p, return_df =  momentum(formation)

In [25]:
# check info in a particular month
return_df[return_df['decile'] == 0].describe()

,2011-04-30 00:00:00,decile
count,74.000000,74.0
mean,-0.041718,0.0
std,0.084471,0.0
min,-0.453885,0.0
25%,-0.047793,0.0
50%,0.000000,0.0
75%,0.000000,0.0
max,0.000000,0.0


In [30]:
# check info in a particular month
return_df[return_df['decile'] == 9].describe()

,2011-04-30 00:00:00,decile
count,51.000000,51.0
mean,0.952800,9.0
std,0.424867,0.0
min,0.647629,9.0
25%,0.709828,9.0
50%,0.835435,9.0
75%,1.040792,9.0
max,3.129608,9.0


In [31]:
SP = yf.download('^GSPC', start=dates[0], end=dates[-1])

[*********************100%***********************]  1 of 1 completed


In [32]:
SP = SP['Adj Close']


In [33]:
SP_monthly_return = SP.pct_change().resample('M').agg(lambda x: (x+1).prod() -1)


In [34]:
SP_monthly_return


Date
2011-05-31    0.000000
2011-06-30   -0.018257
2011-07-31   -0.021474
2011-08-31   -0.056791
2011-09-30   -0.071762
                ...   
2020-12-31    0.037121
2021-01-31   -0.011137
2021-02-28    0.026091
2021-03-31    0.042439
2021-04-30    0.060052
Freq: M, Name: Adj Close, Length: 120, dtype: float64

In [35]:
frame = pd.DataFrame(profits)


In [36]:
frame['S_P500'] = SP_monthly_return.values

In [39]:
frame['excess'] = frame.iloc[:,0] - frame.iloc[:,1]

In [40]:
frame['outperformed'] = ['Yes' if i > 0 else 'No' for i in frame.excess]

In [41]:
frame


,0,S_P500,excess,outperformed
0,0.018539,0.000000,0.018539,Yes
1,-0.005865,-0.018257,0.012393,Yes
2,-0.026081,-0.021474,-0.004606,No
3,-0.072556,-0.056791,-0.015765,No
4,-0.066430,-0.071762,0.005332,Yes
...,...,...,...,...
115,0.004173,0.037121,-0.032949,No
116,-0.096358,-0.011137,-0.085221,No
117,-0.065473,0.026091,-0.091565,No
118,0.019203,0.042439,-0.023236,No


In [42]:
frame[frame.outperformed == "Yes"].shape

(47, 4)

In [43]:
frame.shape

(120, 4)

In [44]:
47/120

0.39166666666666666